In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [20]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [12]:
train_dataset = torchvision.datasets.MNIST(root = '../data',
                                         train = True,
                                         transform = transforms.ToTensor(),
                                         download = True)
test_dataset = torchvision.datasets.MNIST(root = '../data',
                                         train = False,
                                         transform = transforms.ToTensor())


In [13]:
train_loader = torch.utils.data.DataLoader(dataset= train_dataset,
                                          batch_size = batch_size, 
                                          shuffle= True)

test_loader = torch.utils.data.DataLoader(dataset= test_dataset,
                                          batch_size = batch_size, 
                                          shuffle= False)

In [18]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [21]:
criterion =  nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [23]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images =  images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ( i+1) % 100 == 0:
            print("EPoch  [{}/{}], step [{}/{}], loss: {:.4f}".format(epoch+1, num_epochs, i+1, 
                                                                     total_step, loss.item()))

EPoch  [1/5], step [100/600], loss: 0.2827
EPoch  [1/5], step [200/600], loss: 0.3527
EPoch  [1/5], step [300/600], loss: 0.1914
EPoch  [1/5], step [400/600], loss: 0.1852
EPoch  [1/5], step [500/600], loss: 0.1211
EPoch  [1/5], step [600/600], loss: 0.0524
EPoch  [2/5], step [100/600], loss: 0.0883
EPoch  [2/5], step [200/600], loss: 0.2540
EPoch  [2/5], step [300/600], loss: 0.1355
EPoch  [2/5], step [400/600], loss: 0.1391
EPoch  [2/5], step [500/600], loss: 0.0993
EPoch  [2/5], step [600/600], loss: 0.0883
EPoch  [3/5], step [100/600], loss: 0.0850
EPoch  [3/5], step [200/600], loss: 0.0809
EPoch  [3/5], step [300/600], loss: 0.0361
EPoch  [3/5], step [400/600], loss: 0.1299
EPoch  [3/5], step [500/600], loss: 0.1429
EPoch  [3/5], step [600/600], loss: 0.0425
EPoch  [4/5], step [100/600], loss: 0.0267
EPoch  [4/5], step [200/600], loss: 0.0466
EPoch  [4/5], step [300/600], loss: 0.0906
EPoch  [4/5], step [400/600], loss: 0.0246
EPoch  [4/5], step [500/600], loss: 0.0667
EPoch  [4/5

In [26]:
with torch.no_grad():
    correct = 0 
    total = 0 
    
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('accuracy of 1000 test images {} %'.format(100 * correct /total))
    
torch.save(model.state_dict(), 'model.ckpt')

accuracy of 1000 test images 97.82 %
